In [104]:
import pandas as pd
import re
import os
import collections
import nltk
import urllib
import bs4 as bs
import re
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import numpy

from numpy import dot
from numpy.linalg import norm

In [2]:
list_of_files = os.listdir('../top1000_complete')

In [3]:
list_of_files2= []
for i in list_of_files:
    nome = '../top1000_complete/' + i + '/' + 'Documents_xml/' +os.listdir(f'../top1000_complete/{i}/Documents_xml')[0]
    list_of_files2.append(nome)

In [4]:
lines = []
for file in list_of_files2:
    f= open(file,'r')
    lines.append(f.readlines())
    f.close()

In [5]:
paper1= lines[2]

In [6]:
paper1

['<PAPER>\n',
 '  <S sid="0">Advances In Domain Independent Linear Text Segmentation</S>\n',
 '  <ABSTRACT>\n',
 '    <S sid="1" ssid="1">This paper describes a method for linear text segmentation which is twice as accurate and over seven times as fast as the state-of-the-art (Reynar, 1998).</S>\n',
 '    <S sid="2" ssid="2">Inter-sentence similarity is replaced by rank in the local context.</S>\n',
 '    <S sid="3" ssid="3">Boundary locations are discovered by divisive clustering.</S>\n',
 '  </ABSTRACT>\n',
 '  <SECTION title="1 Introduction" number="1">\n',
 '    <S sid="4" ssid="1">Even moderately long documents typically address several topics or different aspects of the same topic.</S>\n',
 '    <S sid="5" ssid="2">The aim of linear text segmentation is to discover the topic boundaries.</S>\n',
 '    <S sid="6" ssid="3">The uses of this procedure include information retrieval (Hearst and Plaunt, 1993; Hearst, 1994; Yaari, 1997; Reynar, 1999), summarization (Reynar, 1998), text un

lista3=[]
lines2=''
for i in range(len(paper1)):
    for j in paper1[i]:
        lines2= lines2+j
    lista3.append(lines2)

In [7]:
texto=''
for i in paper1:
    texto = texto + str(i)

In [213]:
def preprocessa_intro(corpus: str):
    regIter = re.finditer('(<SECTION.*Introduction.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = re.sub(',','', txt)
        txt = re.sub(';','', txt)
        txt = re.sub(':','', txt)
        txt = re.sub('[ ]+',' ', txt)
        txt = re.sub(r'\([^)]*\)', "",txt)
        txt = re.sub(r'\<(.*?)\>',"",txt)
        txt = re.sub(r'small\&',"",txt)
        txt = re.sub(r'tagger&amp',"",txt)
        txt = re.sub(r';quot',"",txt)
        txt = re.sub(r'\&amp',"",txt)
        txt = re.sub(' +', ' ', txt)
        txt = txt.lower()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = txt.strip()
        txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        return txt
    return txt

def preprocessa_abstract(corpus: str):
    regIter = re.finditer("(<ABSTRACT>)(.+?)(</ABSTRACT>)", corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = re.sub(',','', txt)
        txt = re.sub(';',' ;', txt)
        txt = re.sub('[ ]+',' ', txt)
        txt = re.sub(r'\([^)]*\)', "",txt)
        txt = re.sub(r'\<(.*?)\>',"",txt)
        txt = txt.lower()
        txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        lista.append(txt)
    return txt


def preprocessa_conclusion(corpus: str):
    regIter = re.finditer('(<SECTION.*Conclusion.*?>(.*)<\/SECTION>)', corpus, re.DOTALL|re.MULTILINE)
    textos = [ t.group(2) for t in regIter]
    lista = []
    for texto in textos:
        txt = re.sub('\\n',' ', texto)
        txt = re.sub(',','', txt)
        txt = re.sub(';','', txt)
        txt = re.sub(':','', txt)
        txt = re.sub('[ ]+',' ', txt)
        txt = re.sub(r'\([^)]*\)', "",txt)
        txt = re.sub(r'\<(.*?)\>',"",txt)
        txt = re.sub(r'small\&',"",txt)
        txt = re.sub(r'tagger&amp',"",txt)
        txt = re.sub(r';quot',"",txt)
        txt = re.sub(r'\&amp',"",txt)
        txt = re.sub(' +', ' ', txt)
        txt = txt.lower()
        txt = re.sub(' . ', '.', txt).strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        txt = txt.strip()
        #txt = '<sos> ' + txt.strip() + ' <eos>'
        return txt
    return txt

In [214]:
abstract   = preprocessa_abstract(texto)
intro      = preprocessa_intro(texto)
conclusion = preprocessa_conclusion(texto)

In [215]:
token_abstract   = preprocessa_abstract(texto).split()
token_intro      = preprocessa_intro(texto).split()
token_conclusion = preprocessa_conclusion(texto).split()

In [216]:
preprocessa_abstract(texto)

'this paper describes.method for linear text segmentation which is twice as accurate and over seven times as fast as the state-of-the-art.inter-sentence similarity is replaced by rank in the local context. boundary locations are discovered by divisive clustering.'

In [217]:
token_abstract

['this',
 'paper',
 'describes.method',
 'for',
 'linear',
 'text',
 'segmentation',
 'which',
 'is',
 'twice',
 'as',
 'accurate',
 'and',
 'over',
 'seven',
 'times',
 'as',
 'fast',
 'as',
 'the',
 'state-of-the-art.inter-sentence',
 'similarity',
 'is',
 'replaced',
 'by',
 'rank',
 'in',
 'the',
 'local',
 'context.',
 'boundary',
 'locations',
 'are',
 'discovered',
 'by',
 'divisive',
 'clustering.']

In [218]:
tokens = [token_abstract+token_intro+token_conclusion]

In [219]:
model = Word2Vec(tokens, min_count=1)

In [220]:
model.wv.key_to_index

{'the': 0,
 'of': 1,
 'is': 2,
 'and': 3,
 'to': 4,
 'in': 5,
 'for': 6,
 'our': 7,
 'this': 8,
 'similarity': 9,
 'on': 10,
 'segmentation': 11,
 'rank': 12,
 'are': 13,
 'algorithm': 14,
 'text': 15,
 'by': 16,
 'with': 17,
 'that': 18,
 'more': 19,
 'has': 20,
 'we': 21,
 'topic': 22,
 'ranking': 23,
 'which': 24,
 'word': 25,
 'cosine': 26,
 'than': 27,
 'one': 28,
 'measure': 29,
 'boundaries.': 30,
 'from': 31,
 'as': 32,
 'r98': 33,
 'an': 34,
 'two': 35,
 'activation': 36,
 'speed': 37,
 'due': 38,
 'be': 39,
 'clustering': 40,
 'accuracy': 41,
 'spread': 42,
 'described': 43,
 'image': 44,
 'number': 45,
 'it': 46,
 'use': 47,
 'given': 48,
 'method': 49,
 'strategy': 50,
 'accuracy.': 51,
 'scheme': 52,
 'values': 53,
 'segments': 54,
 'lexical': 55,
 'algorithm.': 56,
 'h94': 57,
 'performance': 58,
 'was': 59,
 'algorithms': 60,
 'used': 61,
 'similar': 62,
 'density': 63,
 'my': 64,
 'compare': 65,
 'work': 66,
 'divisive': 67,
 'multi-source': 68,
 'methods': 69,
 'time':

frases_abstract = abstract.split('.')

In [221]:
frases_abstract

['this paper describes',
 ' method for linear text segmentation which is twice as accurate and over seven times as fast as the state-of-the-art',
 ' inter-sentence similarity is replaced by rank in the local context',
 ' boundary locations are discovered by divisive clustering',
 '']

frases_abstract = abstract.split('.')
palavras_frases=[]
for i in frases_abstract:
    palavras_frases.append(i.split(' '))
palavras_frases

In [272]:
def funcao_frase_palavra(abstract):
    """input: lista de frases
        output: lista de palavras de cada frase"""
    frases_abstract = abstract.split('.')
    palavras_frases=[]
    for i in frases_abstract:
        palavras_frases.append(i.split(' '))
    try:
        palavras_frases.remove('')
    except:
        return palavras_frases
    return palavras_frases

In [282]:
def vetor_frases(palavras_frases):
    """input: lista de palavras de cada frase
        output: lista de vetores de cada frase"""
    x=[]
    for i in palavras_frases:
        soma=0
        for j in i:
            try:
                soma = soma +model.wv[j]
            except:
                soma=soma+0
        x.append(soma)
    return x

In [305]:
frases_abstract = abstract.split('.')
palavras_abstract = funcao_frase_palavra(abstract)
try:
    palavras_abstract.remove([''])
except:
    None
vetor_abstract = vetor_frases(palavras_abstract)

In [307]:
frases_intro = intro.split('.').remove('')
palavras_intro = funcao_frase_palavra(intro)
try:
    palavras_intro.remove([''])
except:
    None
vetor_intro = vetor_frases(palavras_intro)

In [332]:
frases_conclusion = conclusion.split('.').remove('')
palavras_conclusion = funcao_frase_palavra(conclusion)
vetor_conclusion = vetor_frases(palavras_conclusion)

In [337]:
h=[]
for i in range(len(vetor_conclusion)):
    x=[]
    for j in range(len(vetor_abstract)):
        x.append(funcao_sim_coseno(vetor_conclusion[i],vetor_abstract[j]))
    h.append(x)

<ipython-input-310-c480b0658c8a>:2: RuntimeWarning: invalid value encountered in true_divide
  cos_sim = dot(a, b)/(norm(a)*norm(b))


In [341]:
h

[[0.11343461, 0.33072603, 0.24621186, -0.013001951],
 [0.13558297, 0.35002366, 0.294813, 0.30642053],
 [0.114904344, 0.2264548, 0.31665838, 0.22883265],
 [0.3164524, 0.48683062, 0.51535493, 0.34803754],
 [0.06724528, 0.08179665, 0.21793294, 0.12674674],
 [0.23069417, 0.50609446, 0.5427517, 0.2817563],
 [0.19042166, 0.38135242, 0.27701235, 0.24419267],
 [0.07874766, 0.20818742, 0.18146408, 0.19055465],
 [0.36211255, 0.5254655, 0.5273954, 0.18882833],
 [0.22723486, 0.62126356, 0.62536186, 0.30707997],
 [0.3635971, 0.6533926, 0.7059151, 0.2644407],
 [0.46812963, 0.558244, 0.57537305, 0.07276361],
 [0.07926293, 0.08658707, 0.104880735, 0.17139836],
 [0.30810407, 0.5897009, 0.67089987, 0.34568548],
 [0.14256617, 0.37697527, 0.37271166, 0.30766088],
 [0.2683056, 0.57055265, 0.4732306, 0.23750886],
 [0.48715988, 0.5753263, 0.5687768, 0.22283491],
 [0.33668265, 0.66536903, 0.57132685, 0.2870629],
 [array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, 

In [ ]:
cosine_similarity = numpy.dot(soma, soma2)/(numpy.linalg.norm(soma)* numpy.linalg.norm(soma2))
cosine_similarity

In [ ]:
import numpy as np

def normalize(word_vec):
    norm=np.linalg.norm(word_vec)
    if norm == 0: 
        return word_vec
    return word_vec/norm

In [ ]:
a=soma
b=soma2

In [ ]:
a=normalize(a)

In [ ]:
b=normalize(b)

In [ ]:
from numpy import dot
from numpy.linalg import norm
cos_sim = dot(a, b)/(norm(a)*norm(b))
cos_sim